## DHTML

In [1]:
from crawler import download
from bs4 import BeautifulSoup
import bs4
import requests
import time
import re
import sqlite3
from html import unescape

이런식으로 어떤 파라메타를 줘서 가져올 것이라면!! form을 줘야한다...

In [2]:
import json

In [46]:
url = "http://example.webscraping.com/places/ajax/search.json"
params = {"search_term": "korea", "page_size": "10", "page": "0"}
html = download("get", url, param=params)

### 개발자도구 > 네트워크 탭을 켜놓은 상태에서 search 버튼 눌러보기
- search json ... 이미지 파일들을 추가로 생성하는게 보임
- html에서 추가되는 파일만 가져오기

In [30]:
data = html.json()

In [31]:
data["records"]

[{'country': 'North Korea',
  'id': 4552797,
  'pretty_link': '<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>'},
 {'country': 'South Korea',
  'id': 4552843,
  'pretty_link': '<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>'}]

In [36]:
c_mark = []
for link in data["records"]:
    print(link["pretty_link"])
    print(link["country"])
    c_mark.append(link["pretty_link"])

<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
North Korea
<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>
South Korea


In [37]:
c_mark

['<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>',
 '<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>']

In [42]:
re.search("<img src=\"(.*)?\"", c_mark[0]).group(1)

'/places/static/images/flags/kp.png'

## Selenium

결국! 자바스크립트를 해석할 수 있게끔 해주는 아이!

브라우저를 띄워서 동적으로 변하는 돔들을 가져올 수 있게끔 해주는 아이

DHTML을 크롤링하거나 파싱할때, 

1. 빠르게 피드백을 하기 위해,
2. 테스팅에서 잘못된 것을 찾아주고,
3. Support for Agile and extreme development methodologies

    - 결국 브라우저를 자동화하기 위해!

In [3]:
from selenium import webdriver

In [9]:
driver = webdriver.Chrome()

네이버 가기 전 아까 것으로 해보기

In [71]:
driver.get("http://example.webscraping.com/places/default/search")

In [59]:
inputTag = driver.find_element_by_id("search_term")
inputTag.send_keys("korea")  ##element를 하나 가지고옴

In [61]:
driver.find_element_by_xpath(".//input[@type='submit']").click()

In [66]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#results")
## 이거는 dom tree로 하는 것!!!

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

요청 url 다르고, dom이 받은 그대로 bs4에 넣어주는 것만으로도 가능!!

눈으로 확인한 Dom을 코드로 그대로 받아서 쓸 수 있는 것!

단점도 존재!! 렌더링 하는데 시간이 존재... 코드로 실행할 시 한번에 하기 때문에~

In [67]:
driver.find_element_by_css_selector("#results").text

'North Korea\nSouth Korea'

### 네이버 selenium으로 DHTML 하기

In [51]:
%%writefile naver_login_info.json
{"id": "pji49990", "pw": "dmswjd10"}

Overwriting naver_login_info.json


In [52]:
with open("naver_login_info.json") as fp:
    account = json.load(fp)

In [53]:
account

{'id': 'pji49990', 'pw': 'dmswjd10'}

In [73]:
driver.get("http://www.naver.com")

In [74]:
driver.find_element_by_css_selector("#account a.lg_local_btn").click()

In [76]:
inputId = driver.find_element_by_id("id")
inputId.send_keys(account["id"])

In [77]:
inputPw = driver.find_element_by_id("pw")
inputPw.send_keys(account["pw"])

In [78]:
driver.find_element_by_xpath(".//input[@class='btn_global']").click()

In [94]:
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print("[CSS]", _.text)

[CSS] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내(재발송)
[CSS] 이용 약관 업데이트
[CSS] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내
[CSS] [데이터진흥원] 데이터 자격시험 응시 최종 확인
[CSS] [데이터진흥원] 데이터 자격시험 응시 수요파악
[CSS] Re:한국의료패널 데이터 활용 동의서입니다.
[CSS] [ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
[CSS] 참여확정 신청 : 2019 빅콘테스트 설명회
[CSS] 박종익 님. 온오프믹스 회원가입을 환영합니다.
[CSS] 고려대_임시_수업자료
[CSS] nashorstyle made you an owner of the koreauniv-bigdata-course organization
[CSS] [RE]RE: [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[CSS] [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[CSS] [GitHub] @nashorstyle has invited you to join the @koreauniv-bigdata-course organization
[CSS] [GitHub] Please verify your device
[CSS] [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[CSS] [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[CSS] 네이버페이 이용약관 개정안내
[CSS] Polaris Office는 당신을 기다립니다!
[CSS] 2019 데이터 청년 캠퍼스 O.T 자료(배포)
[CSS] [빅데이터 청년인재] 고려대학교 과정 - 기본 안내
[CSS] [RE]안녕하세요 교수님! 통계학과 14학번 박종익입니다. 데이터마이닝 성적관련해서 문의드립니다

In [93]:
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[xpath]", _.text)

[xpath] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내(재발송)
[xpath] 이용 약관 업데이트
[xpath] [2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내
[xpath] [데이터진흥원] 데이터 자격시험 응시 최종 확인
[xpath] [데이터진흥원] 데이터 자격시험 응시 수요파악
[xpath] Re:한국의료패널 데이터 활용 동의서입니다.
[xpath] [ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
[xpath] 참여확정 신청 : 2019 빅콘테스트 설명회
[xpath] 박종익 님. 온오프믹스 회원가입을 환영합니다.
[xpath] 고려대_임시_수업자료
[xpath] nashorstyle made you an owner of the koreauniv-bigdata-course organization
[xpath] [RE]RE: [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[xpath] [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[xpath] [GitHub] @nashorstyle has invited you to join the @koreauniv-bigdata-course organization
[xpath] [GitHub] Please verify your device
[xpath] [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[xpath] [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[xpath] 네이버페이 이용약관 개정안내
[xpath] Polaris Office는 당신을 기다립니다!
[xpath] 2019 데이터 청년 캠퍼스 O.T 자료(배포)
[xpath] [빅데이터 청년인재] 고려대학교 과정 - 기본 안내
[xpath] [RE]안녕하세요

In [92]:
print(driver.find_element_by_css_selector("ol.mailList.preview_none").text)

한국데이터산업진흥..
[2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내(재발송)
07-15 10:44
Microsoft
이용 약관 업데이트
07-13 20:56
한국데이터산업진흥..
[2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내
07-12 16:46
선동언
[데이터진흥원] 데이터 자격시험 응시 최종 확인
07-12 11:30
선동언
[데이터진흥원] 데이터 자격시험 응시 수요파악
07-11 14:51
한국의료패널
Re:한국의료패널 데이터 활용 동의서입니다.
07-11 13:36
anonymousUser
[ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
07-10 11:37
온오프믹스
참여확정 신청 : 2019 빅콘테스트 설명회
07-04 13:07
온오프믹스
박종익 님. 온오프믹스 회원가입을 환영합니다.
07-04 13:07
선동언 (Google 드라..
고려대_임시_수업자료
07-01 22:23
koreauniv-bigdata-..
nashorstyle made you an owner of the koreauniv-bigdata-course organization
07-01 19:30
Younghae Do
[RE]RE: [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
07-01 17:37
Younghae Do
[RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
07-01 17:15
GitHub
[GitHub] @nashorstyle has invited you to join the @koreauniv-bigdata-course organization
07-01 16:45
GitHub
[GitHub] Please verify your device
07-01 10:19
Younghae Do
[RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니

In [100]:
driver.find_element_by_css_selector("span.item_wrap.bu6").click()

In [102]:
for cookie in driver.get_cookies():
    print(cookie["name"], cookie["value"])
##쿠키 중에 id값을 가져오겠다~

NIPD 1
NNB 5ZBSGGJYQMXF2
NID_JKL gP6hgtbxZd1EQ/5VM2ChCnJI8PeDn2u2yejvTJ5K47k=
nid_inf 1510623968
NMUSER YXYmFAEwaqbsaxblFq2ZbzGGFAvlaxgsKxErKqKqKAMwFon9KxUmaqgsaqRJaw/qKARpad/syqvstoRVadUstonstonsarRTBdRLa9vstonsH405pzk/7xt5W4d5W4JrpBU5MreR7A2lKAgs16lvpB2RFV9qp6FTW43CbNv5W4d5W4JrpBU5MreR7A2lKAgs
NID_SES AAABeNEFDX5QZWF7u/kpjrOGf8YI3yMOwpNhO7+aOSJrtEEzn2mNYj2BqocCCfmm/xVNojBi6T4OM5jF8urLdbsuybjG6uZDpas/73L/16fZSt9pAIlW1dKS7MiL9YZAxImL7UKoNs5PLRGuqglr7tXx2jMEf60lzbbSquab9XC8NiOOMz1JHaBRXw2EA9zOlu/uxiSqRh7DrM3efOAGfiYtdc9gjjFoD9aU81WK+SxvzpH05lsHznaX0Yv55shwKp2IPtBUCC1Sj/EpVp4FMXg8OxsoqMNmO324kOsWCoZYSNzbD3YywKVTGDXbr1p1NBrC9Jrpa2EEYUQAf4vedFZb9UdQMmyxS7fDtwPgEgT19tr9PJSTqt654wjUAH67AWIEWiYysyVaJSH25kFN/qsnGQ552oLIDTPNPRW4Df73c96rvU3TtRtibbiI+LvNabmWkihsZzOyYrG7pvNJ0Uv7MXPxvoYI9orax48zzAPcBAsCZjP5BkV45mTt18YXz/SPgQ==
NID_AUT iX69FS7h5cZiVAfhC2BS+3r+wvVfDzX6a0yTDQOsjc9GF4kpHvuNYLL7vLcsiPrl


In [103]:
session = requests.Session()

In [104]:
session.cookies  ##당연히 없어야함

<RequestsCookieJar[]>

In [105]:
for cookie in driver.get_cookies():
    session.cookies.set(cookie["name"], cookie["value"])
##쿠키 중에 id값을 가져오겠다~

In [106]:
session.cookies
##이제는 로그인 정보가 들어있다~

<RequestsCookieJar[Cookie(version=0, name='NID_AUT', value='iX69FS7h5cZiVAfhC2BS+3r+wvVfDzX6a0yTDQOsjc9GF4kpHvuNYLL7vLcsiPrl', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='gP6hgtbxZd1EQ/5VM2ChCnJI8PeDn2u2yejvTJ5K47k=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_SES', value='AAABeNEFDX5QZWF7u/kpjrOGf8YI3yMOwpNhO7+aOSJrtEEzn2mNYj2BqocCCfmm/xVNojBi6T4OM5jF8urLdbsuybjG6uZDpas/73L/16fZSt9pAIlW1dKS7MiL9YZAxImL7UKoNs5PLRGuqglr7tXx2jMEf60lzbbSquab9XC8NiOOMz1JHaBRXw2EA9zOlu/uxiSqRh7DrM3efOAGfiYtdc9gjjFoD9aU81WK+SxvzpH05lsHznaX0Yv55shwKp2IPtBU

In [108]:
html = session.get(
    "https://mail.naver.com/#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A%220%22%2C%22folderSN%22%3A%225%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D"
)
html.text
## url은 그냥 스팸메일함~

'\n\n\n<!DOCTYPE HTML>\n\n\t\n\t\n\t\n\t\n\t\n\t\t<html lang="ko">\n\t\n\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190611.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190611.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net" />\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190611.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\

In [113]:
html = session.post(
    "https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=pji49990"
)
for _ in html.json()["mailData"]:
    print(unescape(_["subject"]))

[2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내(재발송)
[2019 데이터 청년 캠퍼스] 취업지원 프로그램 및 자격검정 신청 안내
[데이터진흥원] 데이터 자격시험 응시 최종 확인
[데이터진흥원] 데이터 자격시험 응시 수요파악
Re:한국의료패널 데이터 활용 동의서입니다.
[ 공공데이터포털 ] 회원가입을 위한 이메일 인증번호
참여확정 신청 : 2019 빅콘테스트 설명회
박종익 님. 온오프믹스 회원가입을 환영합니다.
고려대_임시_수업자료
nashorstyle made you an owner of the koreauniv-bigdata-course organization
[RE]RE: [RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[RE]RE: [RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[GitHub] @nashorstyle has invited you to join the @koreauniv-bigdata-course organization
[GitHub] Please verify your device
[RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
[RE]안녕하세요 도영해 교수님! 저는 이번 수리적 빅데이터 과목을 수강했던 통계학과 박종익입니다.
네이버페이 이용약관 개정안내
Polaris Office는 당신을 기다립니다!
2019 데이터 청년 캠퍼스 O.T 자료(배포)
[빅데이터 청년인재] 고려대학교 과정 - 기본 안내
[RE]안녕하세요 교수님! 통계학과 14학번 박종익입니다. 데이터마이닝 성적관련해서 문의드립니다...
[TIDE #6] 미래를 준비하는 타이드인스티튜트
[mgram] 지난번 진단으로부터 9개월이 경과했습니다(지난번: 2018년9월24일)
'2019 데이터 청년 캠퍼스' 오리엔테이션 참석 확인 및 온라인 교안 제공  
주문해 주셔서 감사합니다
'201

셀레니움가지고 데이터를 가져오면 안된다~ 왜냐하면 렌더링의 시간이 걸리기때문에!!

그리고 이제부터 오류를 보여줄 것!

In [114]:
html = requests.post(
    "https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=pji49990"
)
html.text

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

똑같은 방식으로 했지만! 셀레니움은 쿠키값이 존재하기 때문에 로그인이 된다~

Ajax로 된 것들은 무조건 url이 존재하긴 한다. 그 url을 얼마나 잘 찾을 것인지가 보임!

In [116]:
html = session.post(
    "https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=pji49990"
)
for _ in html.json()["mailData"]:
    print(unescape(_["subject"]))

이용 약관 업데이트


In [131]:
driver.get("https://section.cafe.naver.com/")

In [134]:
for _ in driver.find_elements_by_css_selector(
        "div.user_mycafe_list strong.name"):
    print("[CSS]", _.text)

for _ in driver.find_elements_by_xpath(
        "//div[@class='user_mycafe_list']//a/strong[@class='name']"):
    print("[xpath]", _.text)

[CSS] Crawling Test Cafe
[CSS] 독취사-취업,대학생,대기업,공기업,NCS,인턴,공모전,승무원,토익
[CSS] 스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
[CSS] 렛유인｜이공계취업-반도체공정교육,컨설팅,자소서첨삭,직무
[CSS] MEL PARK's TOEIC SPEAKING
[CSS] 통계와조사나라 (SAS/SPSS/R/Stata/마케팅/논문통계/통계대학원)
[CSS] 더 콥스 - Liverpool Football Club
[CSS] 초록우산어린이재단 반디 인 대구
[CSS] 이탈리아 청춘민박
[CSS] 경북대학교 미담장학회
[xpath] Crawling Test Cafe
[xpath] 독취사-취업,대학생,대기업,공기업,NCS,인턴,공모전,승무원,토익
[xpath] 스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
[xpath] 렛유인｜이공계취업-반도체공정교육,컨설팅,자소서첨삭,직무
[xpath] MEL PARK's TOEIC SPEAKING
[xpath] 통계와조사나라 (SAS/SPSS/R/Stata/마케팅/논문통계/통계대학원)
[xpath] 더 콥스 - Liverpool Football Club
[xpath] 초록우산어린이재단 반디 인 대구
[xpath] 이탈리아 청춘민박
[xpath] 경북대학교 미담장학회


In [126]:
html = session.get(
    "https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=10&articleCount=3&useMyCafeEvent=true"
)
html.json()

{'message': {'error': {'code': '', 'msg': ''},
  'result': {'badgeCounts': {'activityCount': 0,
    'alarmCount': {'generalAlarmCount': 0, 'marketAlarmCount': 0},
    'cafeTalkCount': 0},
   'discoveryTab': {'discoveryTabs': [{'themeId': 102, 'themeName': '추천카페'},
     {'themeId': 14, 'themeName': '건강/다이어트'},
     {'themeId': 3, 'themeName': '방송/연예'},
     {'themeId': 11, 'themeName': '취미'},
     {'themeId': 2, 'themeName': '만화/애니'},
     {'themeId': 24, 'themeName': '친목/모임'},
     {'themeId': 1, 'themeName': '게임'},
     {'themeId': 25, 'themeName': '종교/봉사'},
     {'themeId': 10, 'themeName': '반려동물/동물'},
     {'themeId': 9, 'themeName': '스포츠/레저'},
     {'themeId': 4, 'themeName': '문화/예술'},
     {'themeId': 20, 'themeName': '경제/금융'},
     {'themeId': 5, 'themeName': '영화'},
     {'themeId': 22, 'themeName': '문학/창작'},
     {'themeId': 16, 'themeName': '컴퓨터/통신'},
     {'themeId': 13, 'themeName': '패션/미용'},
     {'themeId': 18, 'themeName': '외국어'},
     {'themeId': 8, 'themeName': '여행'},
  

In [127]:
for _ in html.json()["message"]["result"]["myCafe"]["cafes"]:
    print(_["cafeName"])

Crawling Test Cafe
독취사-취업,대학생,대기업,공기업,NCS,인턴,공모전,승무원,토익
스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
렛유인｜이공계취업-반도체공정교육,컨설팅,자소서첨삭,직무
MEL PARK's TOEIC SPEAKING
통계와조사나라 (SAS/SPSS/R/Stata/마케팅/논문통계/통계대학원)
더 콥스 - Liverpool Football Club
초록우산어린이재단 반디 인 대구
이탈리아 청춘민박
경북대학교 미담장학회


페이지를 띄워서 로그아웃을 해야하는데, 그 시간을 없애버린다...

In [148]:
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()
## + 는 내 바로 다음 형제!

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [149]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#gnb_name2 + a.gnb_btn_login")
##하지만 존재함!!!!

[<a class="gnb_btn_login" href="https://nid.naver.com/nidlogin.logout?returl=https://section.cafe.naver.com/Logout.nhn?alreadyNaverLogout=true" id="gnb_logout_button"><span class="gnb_bg"></span><span class="gnb_bdr"></span><span class="gnb_txt">로그아웃</span></a>]

네이버 뉴스에서 헤드라인 뉴스들 수집할 것인데, 30개에 대한 내용을 가지고 올 것이다!

In [150]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "html.parser")

    return [
        requests.compat.urljoin(seed,
                                tag.find_parent()["href"])
        for tag in dom.select("font.list_title")
        if tag.find_parent() and tag.find_parent().has_attr("href")
    ]

In [151]:
def parseContent(url):
    html = download("get", url)
    dom = BeautifulSoup(html.text, "html.parser")

    return {
        "title":
        dom.select_one("font.view_title2").text.strip(),
        "body":
        dom.select_one("table.pic_bg td.han").text.strip(),
        "comments": [
            tag.text.strip("table.pic_bg td.han")
            for tag in dom.select("div[id^=commentContent_]")
        ]
    }

In [188]:
driver.get("https://news.naver.com/")

'https://news.naver.com/'

In [200]:
[
    requests.compat.urljoin(driver.current_url, _["href"])
    for _ in dom.select("#main_content a")
    if "main/read.nhn" in _["href"] and "clusterArticles" not in _["href"]
]

['https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=081&aid=0003014740',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0010959390',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=081&aid=0003014739',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=056&aid=0010722920',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=015&aid=0004177135',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=119&aid=0002341108',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=422&aid=0000383920',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=448&aid=0000278107',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=448&aid=0000278107',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=437&aid=0000215223',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=008&aid=0004249914',
 'https://

In [196]:
dom = BeautifulSoup(driver.page_source, "lxml")
urlList = [
    requests.compat.urljoin(url, _["href"])
    for _ in dom.select("#main_content a") if _.has_attr("href")
    and "main/read.nhn" in _["href"] and "clusterArticles" not in _["href"]
    and "javascript:" not in _["href"] and len(_["href"]) > 3
]

In [198]:
len(urlList)

49

In [208]:
body = []

In [207]:
driver.get(urlList[0])
body.append(driver.find_element_by_css_selector("#articleBodyContents").text)

In [215]:
title = []
driver.find_element_by_css_selector("#articleTitle").text

"리브라 청문회…페북이 '답한 것'과 '답하지 않은 것'"

In [11]:
body = []
for url in urlList:
    driver.get(url)
    body.append(
        driver.find_element_by_css_selector("#articleBodyContents").text)

NameError: name 'urlList' is not defined

In [5]:
con = sqlite3.connect("NaverNEWS.db")

In [6]:
cur = con.cursor()

In [10]:
cur.executescript('''
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1 (
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        url TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
        );
''')

cur.executescript('''
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2 (
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        table1_id INTEGER NOT NULL,
        title Text NOT NULL);
''')
con.commit()

In [ ]:
dom = BeautifulSoup(driver.page_source, "lxml")
urlList = [
    requests.compat.urljoin(url, _["href"])
    for _ in dom.select("#main_content a") if _.has_attr("href")
    and "main/read.nhn" in _["href"] and "clusterArticles" not in _["href"]
    and "javascript:" not in _["href"] and len(_["href"]) > 3
]

In [ ]:
title = []
driver.find_element_by_css_selector("#articleTitle").text

In [ ]:
body = []
for url in urlList:
    driver.get(url)
    body.append(
        driver.find_element_by_css_selector("#articleBodyContents").text)

In [219]:
urlList = [
    requests.compat.urljoin(url, _["href"])
    for _ in dom.select("#main_content a") if _.has_attr("href")
    and "main/read.nhn" in _["href"] and "clusterArticles" not in _["href"]
    and "javascript:" not in _["href"] and len(_["href"]) > 3
]

for i in range(len(urlList)):
    driver.get(urlList[i])
    title = driver.find_element_by_css_selector("#articleTitle").text
    content = driver.find_element_by_css_selector("#articleBodyContents").text
    url = urlList[i]
    cur.execute(
        """
        INSERT INTO table1
        (title, content, url)
        VALUES(?, ?, ?);
        """, [title, content, url])
    con.commit()

내가 한 것은 수집을 위한 것

아까 네이버 로그아웃을 봤던 건 element가 있는데, 접근이 안되는 것이고,

두번째 오류는 렌더링의 문제!!

In [119]:
cur.executescript('''
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1 (
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
        );
''')

cur.executescript('''
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2 (
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        table1_id INTEGER NOT NULL,
        content Text NOT NULL);
''')
con.commit()

In [48]:
driver.get("https://news.naver.com")
N = len([ _ for _ in driver.find_elements_by_css_selector(
    ".hdline_news .hdline_article_tit > a, .com_list li > a")
         if len(_.get_attribute("href")) > 1])

In [49]:
for i in range(N):
    driver.find_element_by_css_selector("#lnb > ul > li:first-child > a").click()
    news = [_ for _ in 
           driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a")
           if len (_.get_attribute("href")) > 1][i]
    cur.execute("Insert into table1(title, url) values(?,?)", [news.text, news.get_attribute("href")])
    con.commit()
    
    cur.execute("SELECT id FROM table1 WHERE url=?",[news.get_attribute("href")])
    newsID = cur.fetchone()[0]
    
    
    news.click()
    
    cur.execute("insert into table2(table1_id, content) values(?,?)",
               [newsID, driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()

In [17]:
driver.get("https://mail.naver.com/")

In [24]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [28]:
wait = WebDriverWait(driver, 10, 0.5, 
              [ElementNotInteractableException])
##어떻게 보면 try except 구문을 빠져나가지 말고 이렇게 하라~
wait.until(EC.visibility_of_element_located(
    (By.CSS_SELECTOR, "#gnb_name2 + a.gnb_btn_login")))
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()
##보이는 것을 숨기려면 until_not이고, 반대는 until
##하나의 element를 볼 것이다~ 그리고 신경쓸 것들은 visibility

TimeoutException: Message: 


In [37]:
wait = WebDriverWait(driver, 10, 0.5, 
              [ElementNotInteractableException])
wait.until(
    lambda x:x.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").
    is_displayed())
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

로그아웃 버튼이 보여야 하는데 지금은 안보이니까

클릭을 해서 보이게 하는 순간 로그아웃이 된다.

In [38]:
logout = driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


### iframe 불러오는 법!

In [70]:
driver.get("https://www.naver.com/")

In [71]:
iframe = driver.find_element_by_name("da_iframe_time")

In [72]:
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


여기서 dom 안의 dom으로 들어옴!

In [73]:
driver.find_element_by_tag_name("img").get_attribute("src")

'https://ssl.pstatic.net/tveta/libs/1242/1242461/f4c778fe28e51ecc6ee2_20190715175440120.jpg'

In [74]:
driver.switch_to_default_content

<bound method WebDriver.switch_to_default_content of <selenium.webdriver.chrome.webdriver.WebDriver (session="17c9c2929ac0cc47b60b0b91d4a13587")>>

In [76]:
driver.window_handles

['CDwindow-AFE8FB0B1B9C923441AF6DE86A14B84E',
 'CDwindow-FCA9AA1881CFECD86F49952B811C670D']

이거는 떠있는 창들 보는 것! tab이 되어 있으면 두개씩 or 더 뜬다~

In [77]:
#driver.switch_to_window()
# 이 친구는 그 창으로 가라~

In [96]:
driver.get("https://www.kt.com/")

In [97]:
driver.find_element_by_css_selector("span.quick > a").click()

In [108]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [99]:
driver.find_element_by_id("userId").send_keys("test")

In [102]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="72fd16e4d4c724aafc5f20cbf1bc2cc2")>

In [105]:
from selenium.webdriver.common.keys import Keys

In [106]:
Keys.CONTROL+"n"

'\ue009n'

In [112]:
driver.execute_script("javascript:window.open('about:blank');")

In [118]:
driver.switch_to_window(driver.window_handles[1])
driver.get("http://www.naver.com")
driver.switch_to_window(driver.window_handles[0])

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.
C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.window instead
  This is separate from the ipykernel package so we can avoid doing imports until


keys는 웹드라이브에다가 키 이벤트 주는거

In [101]:
driver.close()

In [95]:
driver = webdriver.Chrome()

In [ ]:
driver.execute_script("javascript:window.open('about:blank');")
driver.switch_to_window(driver.window_handles[1])
driver.get("http://www.naver.com")
driver.switch_to_window(driver.window_handles[0])

In [125]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [126]:
driver.get("https://news.naver.com")
N = [ _ for _ in driver.find_elements_by_css_selector(
    ".hdline_news .hdline_article_tit > a, .com_list li > a")
         if len(_.get_attribute("href")) > 1]

In [127]:
for _ in [_ for _ in 
       driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a")
       if len (_.get_attribute("href")) > 1]:
    news = (_.text, _.get_attribute("href"))
    
    cur.execute("Insert into table1(title, url) values(?,?)", [news[0], news[1]])
    
    con.commit()
    
    cur.execute("SELECT id FROM table1 WHERE url=?",[news[1]])
    newsID = cur.fetchone()[0]
    
    driver.execute_script("javascript:window.open('about:blank');")
    driver.switch_to_window(driver.window_handles[-1])
    driver.get(news[1])
    
    cur.execute("insert into table2(table1_id, content) values(?,?)",
               [newsID, driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()
    
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    
driver.close()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.window instead
